---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [ ]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [ ]:
def answer_one():
    return 100*(len(spam_data[spam_data['target']==1])/len(spam_data))

In [ ]:
answer_one()

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vec = CountVectorizer().fit(X_train)
    names = sorted(vec.get_feature_names(), key=lambda x:len(x), reverse=True)
    
    return names[0]

In [ ]:
answer_two()

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

vect = CountVectorizer().fit(X_train)
X_train_vectorized = vect.fit_transform(X_train)
model = MultinomialNB(alpha=0.1)
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
def answer_three():
    return roc_auc_score(y_test, predictions, average='micro')


In [ ]:
answer_three()

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector = TfidfVectorizer().fit(X_train)
vect_features = vector.transform(X_train)
names = np.asarray(vector.get_feature_names())
values = vect_features.max(0).toarray()[0]
index = vect_features.max(0).toarray()[0].argsort()
smallest = pd.Series(values[index[:20]], index=names[index[:20]])
smallest.sort_index(inplace=True)

highest = pd.Series(values[index[-20:]], index=names[index[-20:]])
highest.sort_index(inplace=True)
def answer_four():
    return (smallest, highest)

In [ ]:
answer_four()

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
vector = TfidfVectorizer(min_df=3).fit(X_train)
X_test_vectorized = vector.transform(X_test)
X_train_vectorized = vector.transform(X_train)
model = MultinomialNB(alpha=0.1)
model.fit(X_train_vectorized, y_train)
predictions = model.predict(X_test_vectorized)  
def answer_five():   
    return roc_auc_score(y_test, predictions, average='micro')


In [ ]:
answer_five()

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
spam_data['taille'] = spam_data['text'].map(lambda x:len(x))
def answer_six():
    no_spam = spam_data[spam_data['target']==0]['taille'].mean()
    spam = spam_data[spam_data['target']==1]['taille'].mean()
    
    return (no_spam, spam)

In [ ]:
answer_six()

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [ ]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.svm import SVC
model = SVC(C=10000)

X_train_vectorized = vector_ter.transform(X_train)
X_train_frame = pd.DataFrame(X_train, columns=['text'])
taille_X_train = X_train_frame['text'].map(lambda x:len(word_tokenize(x))).values
X_vectorized = add_feature(X_train_vectorized, taille_X_train)


X_test_vecto = vector_ter.transform(X_test)
X_test_frame = pd.DataFrame(X_test, columns=['text'])
taille_X_test = X_test_frame['text'].map(lambda x:len(word_tokenize(x))).values
X_test_vecto = add_feature(X_test_vecto, taille_X_test)


model.fit(X_vectorized, y_train)
prediction = model.predict(X_test_vecto)
def answer_seven():
    return roc_auc_score(y_test, prediction, average='micro')


In [ ]:
answer_seven()

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [ ]:
import re
re_exp = re.compile('\d')
def compute_digit(text):
    result = re_exp.findall(text)
    if len(result)==0:
        return 0
    else:
        return len(result)

spam_data['number_digit'] = spam_data['text'].map(compute_digit)

In [ ]:
def answer_eight(): 
    no_spam = spam_data[spam_data['target']==0]['number_digit'].mean()
    spam = spam_data[spam_data['target']==1]['number_digit'].mean()
    
    return (no_spam, spam)

In [ ]:
answer_eight()

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=100)

vector_ter = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)

X_train_vectorized = vector_ter.transform(X_train)
X_train_frame = pd.DataFrame(X_train, columns=['text'])
taille_X_train = X_train_frame['text'].map(lambda x:len(word_tokenize(x))).values
digit_X_train = X_train_frame['text'].map(compute_digit).values
X_vectorized = add_feature(X_train_vectorized, taille_X_train)
X_vectorized = add_feature(X_vectorized, digit_X_train)

X_test_vecto = vector_ter.transform(X_test)
X_test_frame = pd.DataFrame(X_test, columns=['text'])
taille_X_test = X_test_frame['text'].map(lambda x:len(word_tokenize(x))).values
digit_X_test = X_test_frame['text'].map(compute_digit).values
X_test_vecto = add_feature(X_test_vecto, taille_X_test)
X_test_vecto = add_feature(X_test_vecto, digit_X_test)

model.fit(X_vectorized, y_train)
prediction = model.predict(X_test_vecto)
def answer_nine():
      
    return roc_auc_score(y_test, prediction, average='micro')


In [ ]:
answer_nine()

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [ ]:
re_exp = re.compile('\W')
def compute_non_digit(text):
    result = re_exp.findall(text)
    if len(result)==0:
        return 0
    else:
        return len(result)

spam_data['no_digit'] = spam_data['text'].map(compute_non_digit)
def answer_ten():
    no_spam = spam_data[spam_data['target']==0]['no_digit'].mean()
    spam = spam_data[spam_data['target']==1]['no_digit'].mean()
    
    return (no_spam, spam)

In [ ]:
answer_ten()

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [ ]:
model = LogisticRegression(C=100)

vector = TfidfVectorizer(min_df=5, ngram_range=(2,5)).fit(X_train)

X_train_vectorized = vector.transform(X_train)
X_train_frame = pd.DataFrame(X_train, columns=['text'])
taille_X_train = X_train_frame['text'].map(lambda x:len(word_tokenize(x))).values
digit_X_train = X_train_frame['text'].map(compute_digit).values
no_digit_X_train = X_train_frame['text'].map(compute_non_digit).values
X_vectorized = add_feature(X_train_vectorized, taille_X_train)
X_vectorized = add_feature(X_vectorized, digit_X_train)
X_vectorized = add_feature(X_vectorized, no_digit_X_train)

X_test_vecto = vector.transform(X_test)
X_test_frame = pd.DataFrame(X_test, columns=['text'])
taille_X_test = X_test_frame['text'].map(lambda x:len(word_tokenize(x))).values
digit_X_test = X_test_frame['text'].map(compute_digit).values
no_digit_X_test = X_test_frame['text'].map(compute_non_digit).values
X_test_vecto = add_feature(X_test_vecto, taille_X_test)
X_test_vecto = add_feature(X_test_vecto, digit_X_test)
X_test_vecto = add_feature(X_test_vecto, no_digit_X_test)

model.fit(X_vectorized, y_train)
prediction = model.predict(X_test_vecto)

features = vector.get_feature_names()+ ['length_of_doc', 'digit_count', 'non_word_char_count']
features = np.asarray(features)
importance = model.coef_[0].argsort()
def answer_eleven():

    return (roc_auc_score(y_test, prediction, average='micro'), features[importance[:20]], features[importance[-20:]])

In [ ]:
answer_eleven()